In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd


In [0]:
train=pd.read_csv('/content/gdrive/My Drive/train.csv')
test=pd.read_csv('/content/gdrive/My Drive/test.csv')
test.head()
train.head()

,id,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [0]:
y = train['type']
X = train.drop(columns=['type'], axis=1)

In [0]:
from xgboost import XGBClassifier
xgb=XGBClassifier(n_estimators=2000,scoring='log_loss',
                        n_jobs=4,
                        max_depth=15,
                        learning_rate=0.05,
                        gamma = 0.02,
                        subsample = 0.9,
                        colsample_bytree=0.9,
                        tree_method='gpu_hist')

In [0]:
xgb.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.02,
              learning_rate=0.05, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=2000, n_jobs=4,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, scoring='log_loss',
              seed=None, silent=None, subsample=0.9, tree_method='gpu_hist',
              verbosity=1)

In [0]:
id_for_index = test['id']
types = train['type'].unique()
testing = test[test.columns[1:]]

In [0]:
predictions = xgb.predict_proba(test)
print(predictions)

[[6.28008365e-07 3.24949397e-05 3.24473149e-07 ... 9.94755030e-01
  5.42708949e-06 3.38445858e-08]
 [3.76671014e-05 9.54536378e-01 1.71001795e-06 ... 4.94930646e-07
  1.74024308e-06 2.30887531e-06]
 [9.99850750e-01 6.12170334e-06 2.03111995e-07 ... 1.76364381e-08
  1.67047887e-08 1.58493805e-07]
 ...
 [6.88135637e-09 9.99998808e-01 1.26283455e-08 ... 3.19797244e-10
  6.60353494e-10 3.63448258e-08]
 [1.30475986e-08 9.99999404e-01 5.75886583e-09 ... 3.17806159e-09
  1.71059455e-09 7.54162599e-09]
 [4.07246468e-08 9.40813720e-07 9.82977788e-09 ... 1.97272421e-09
  5.45159518e-09 4.71416897e-08]]


In [0]:
result = pd.DataFrame(data=predictions, index=id_for_index, columns=xgb.classes_)
result.head()

,GALAXY,QSO,REDDEN_STD,ROSAT_D,SERENDIPITY_BLUE,SERENDIPITY_DISTANT,SERENDIPITY_FIRST,SERENDIPITY_MANUAL,SERENDIPITY_RED,SKY,SPECTROPHOTO_STD,STAR_BHB,STAR_BROWN_DWARF,STAR_CARBON,STAR_CATY_VAR,STAR_PN,STAR_RED_DWARF,STAR_SUB_DWARF,STAR_WHITE_DWARF
id,,,,,,,,,,,,,,,,,,,
199991,6.280084e-07,0.000032,3.244731e-07,0.000190,3.354513e-06,2.499290e-06,1.526882e-06,1.633180e-05,4.979850e-03,1.572567e-07,6.910791e-08,1.255549e-08,1.172293e-05,1.443208e-07,1.194186e-07,1.895503e-07,9.947550e-01,5.427089e-06,3.384459e-08
199992,3.766710e-05,0.954536,1.710018e-06,0.037289,7.524014e-06,4.926397e-05,8.048583e-03,3.842245e-06,3.059936e-06,9.269054e-07,9.584211e-07,6.108912e-06,3.657794e-07,2.621434e-06,2.816790e-06,4.383131e-06,4.949306e-07,1.740243e-06,2.308875e-06
199993,9.998507e-01,0.000006,2.031120e-07,0.000139,5.082165e-08,4.093702e-08,2.740164e-06,7.849672e-08,1.055368e-08,1.023122e-08,2.154590e-08,3.019601e-08,1.489259e-08,9.042279e-08,9.843666e-08,2.477213e-08,1.763644e-08,1.670479e-08,1.584938e-07
199994,2.105414e-07,0.003157,9.479702e-07,0.000245,9.962320e-01,4.652842e-05,3.095058e-04,1.797158e-06,2.108889e-07,2.683670e-07,3.986445e-07,4.041445e-07,3.981922e-08,6.857401e-08,1.298608e-06,3.881629e-06,3.459831e-08,6.179886e-08,2.349353e-07
199995,1.287090e-07,0.000003,1.143571e-07,0.000057,7.955018e-08,4.943247e-08,2.969104e-07,1.929497e-07,6.532188e-07,5.375778e-09,4.017680e-09,4.940738e-09,1.916359e-08,8.206066e-08,3.463509e-09,9.882630e-09,9.999392e-01,1.259776e-08,5.259297e-09


In [0]:
sample = pd.read_csv("/content/gdrive/My Drive/sample_submission.csv")
col_order = sample.columns[1:]

In [0]:
result = result[col_order]
result.head()

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
199991,3.384459e-08,1.194186e-07,1.172293e-05,4.979850e-03,3.244731e-07,1.255549e-08,6.280084e-07,2.499290e-06,0.000032,1.572567e-07,9.947550e-01,0.000190,1.895503e-07,1.526882e-06,1.443208e-07,6.910791e-08,5.427089e-06,1.633180e-05,3.354513e-06
199992,2.308875e-06,2.816790e-06,3.657794e-07,3.059936e-06,1.710018e-06,6.108912e-06,3.766710e-05,4.926397e-05,0.954536,9.269054e-07,4.949306e-07,0.037289,4.383131e-06,8.048583e-03,2.621434e-06,9.584211e-07,1.740243e-06,3.842245e-06,7.524014e-06
199993,1.584938e-07,9.843666e-08,1.489259e-08,1.055368e-08,2.031120e-07,3.019601e-08,9.998507e-01,4.093702e-08,0.000006,1.023122e-08,1.763644e-08,0.000139,2.477213e-08,2.740164e-06,9.042279e-08,2.154590e-08,1.670479e-08,7.849672e-08,5.082165e-08
199994,2.349353e-07,1.298608e-06,3.981922e-08,2.108889e-07,9.479702e-07,4.041445e-07,2.105414e-07,4.652842e-05,0.003157,2.683670e-07,3.459831e-08,0.000245,3.881629e-06,3.095058e-04,6.857401e-08,3.986445e-07,6.179886e-08,1.797158e-06,9.962320e-01
199995,5.259297e-09,3.463509e-09,1.916359e-08,6.532188e-07,1.143571e-07,4.940738e-09,1.287090e-07,4.943247e-08,0.000003,5.375778e-09,9.999392e-01,0.000057,9.882630e-09,2.969104e-07,8.206066e-08,4.017680e-09,1.259776e-08,1.929497e-07,7.955018e-08


In [0]:
result.to_csv('/content/gdrive/My Drive/xgb_result.csv', sep=',')